In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mnist-dataset/train-images.idx3-ubyte
/kaggle/input/mnist-dataset/t10k-labels.idx1-ubyte
/kaggle/input/mnist-dataset/t10k-images.idx3-ubyte
/kaggle/input/mnist-dataset/train-labels.idx1-ubyte
/kaggle/input/mnist-dataset/t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte
/kaggle/input/mnist-dataset/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte
/kaggle/input/mnist-dataset/train-labels-idx1-ubyte/train-labels-idx1-ubyte
/kaggle/input/mnist-dataset/train-images-idx3-ubyte/train-images-idx3-ubyte


In [6]:
!pip install torch torchvision numpy idx2numpy -q

  Preparing metadata (setup.py) ... done


In [20]:
import idx2numpy
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim

In [8]:
# Load training images and labels
train_images = idx2numpy.convert_from_file('/kaggle/input/mnist-dataset/train-images-idx3-ubyte/train-images-idx3-ubyte')
train_labels = idx2numpy.convert_from_file('/kaggle/input/mnist-dataset/train-labels-idx1-ubyte/train-labels-idx1-ubyte')

# Load test images and labels
test_images = idx2numpy.convert_from_file('/kaggle/input/mnist-dataset/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte')
test_labels = idx2numpy.convert_from_file('/kaggle/input/mnist-dataset/t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte')

In [9]:
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [14]:
train_images_tensor = torch.tensor(train_images, dtype=torch.float32).unsqueeze(1)
train_labels_tensor = torch.tensor(train_labels, dtype = torch.int64)

test_images_tensor = torch.tensor(test_images,dtype = torch.float32).unsqueeze(1)
test_labels_tensor = torch.tensor(test_labels, dtype = torch.int64)

In [16]:
#create tensordataset
train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)

test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)

In [18]:
#create Dataloader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle = False)

#chheck a batch of data

for images,labels in train_loader:
    print("batch of images shape",images.shape)
    print("batch of labels:", labels)
    break

batch of images shape torch.Size([32, 1, 28, 28])
batch of labels: tensor([1, 6, 2, 1, 0, 9, 0, 3, 2, 9, 4, 1, 2, 7, 0, 1, 3, 1, 8, 5, 2, 8, 8, 4,
        7, 8, 3, 2, 5, 0, 8, 4])


In [19]:
train_images_tensor = train_images_tensor / 255.0
test_images_tensor = test_images_tensor/255.0

In [31]:

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1=nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1)
        self.pool =nn.MaxPool2d(kernel_size=2,stride=2,padding=0)
        self.fc1=nn.Linear(32*14*14,10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(-1,32*14*14)
        x = self.fc1(x)
        return x

In [34]:
train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Initialize the model, loss function, and optimizer
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0009)

In [35]:
for epoch in range(5):
    for images,labels in train_loader:
        #fw pass
        outputs=model(images)
        loss=criterion(outputs,labels)

        #bw pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/5], Loss: {loss.item():.4f}")

Epoch [1/5], Loss: 0.0395
Epoch [2/5], Loss: 0.0265
Epoch [3/5], Loss: 0.0434
Epoch [4/5], Loss: 0.0037
Epoch [5/5], Loss: 0.0122


In [37]:
model.eval()  # Set the model to evaluation mode
test_correct = 0
test_total = 0

with torch.no_grad():  # Disable gradient calculation
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 63.53%
